# 0. packages, data

In [2]:
import gurobipy as gp
from gurobipy import GRB

import numpy as np
import pandas as pd
from scipy.spatial import distance
from sklearn.metrics.pairwise import haversine_distances
from math import radians


# 1. Variables

In [3]:
# 수요 포인트 I

I = {'수요1':([125, 36], 10), '수요2':([153, 39], 40), '수요3':([102, 33], 20), 
     '수요4':([111, 31], 20), '수요5':([132, 36], 30), '수요6':([199, 33], 10), 
     '수요7':([110, 30], 30), '수요8':([100, 35], 50), '수요9':([122, 35], 30), '수요10':([131, 31], 10)}
I

{'수요1': ([125, 36], 10),
 '수요2': ([153, 39], 40),
 '수요3': ([102, 33], 20),
 '수요4': ([111, 31], 20),
 '수요5': ([132, 36], 30),
 '수요6': ([199, 33], 10),
 '수요7': ([110, 30], 30),
 '수요8': ([100, 35], 50),
 '수요9': ([122, 35], 30),
 '수요10': ([131, 31], 10)}

In [4]:
# 입지 후보지 J

J = {'강릉': ([122, 29], 0), '춘천': ([101, 28], 0), '진주' : ([100, 31], 0), '구미':([132, 36], 0)}
J

{'강릉': ([122, 29], 0),
 '춘천': ([101, 28], 0),
 '진주': ([100, 31], 0),
 '구미': ([132, 36], 0)}

In [5]:
# 기존 야구장 R

R = {'잠실구장': ([125, 30], 0), '부산구장': ([121, 40], 0), '인천구장': (
    [164, 30], 0), '대구구장': ([177, 38], 0), '광주구장': ([100, 40], 0)}
R

{'잠실구장': ([125, 30], 0),
 '부산구장': ([121, 40], 0),
 '인천구장': ([164, 30], 0),
 '대구구장': ([177, 38], 0),
 '광주구장': ([100, 40], 0)}

In [6]:
# 입지 후보지 j에서 수요 포인트 i까지의 거리 D

d_ji = np.array([])
for j_key, j_value in J.items():
    for i_key, i_value in I.items():
        # d = distance.euclidean(j_value[0], i_value[0])
        d = haversine_distances([[radians(_) for _ in j_value[0]], [
                                radians(_) for _ in i_value[0]]]) * 6371000/1000
        d = d[0][1]
        d_ji = np.append(d_ji, d)
d_ji = d_ji.reshape(len(J), len(I))

D = d_ji
pd.DataFrame(D)

,0,1,2,3,4,5,6,7,8,9
0,543.462518,3534.772371,2228.892498,1227.000864,1205.063865,8569.988731,1335.184726,2454.847349,353.429798,1009.342977
1,2685.312841,5807.351310,156.908277,1115.382239,3462.382120,10901.519879,1002.372116,180.042364,2348.455815,3338.033136
2,2785.581285,5905.334427,226.369198,1223.144193,3563.549903,11008.942905,1112.281091,77.219974,2450.097809,3447.042726
3,778.364487,2349.576328,3338.276378,2351.262149,0.000000,7456.059982,2467.522701,3558.450408,1113.928920,384.730888


In [7]:
# 입지 후보지 j가 수요 포인트 i의 고려 수요 대상인지 나타내는 이진법 변수 E

thresh = 3000  # U_j
e_ji = np.select([d_ji > thresh, d_ji <= thresh], [
                 np.zeros_like(d_ji), np.ones_like(d_ji)])

E = e_ji
E

array([[1., 0., 1., 1., 1., 0., 1., 1., 1., 1.],
       [1., 0., 1., 1., 0., 0., 1., 1., 1., 0.],
       [1., 0., 1., 1., 0., 0., 1., 1., 1., 0.],
       [1., 1., 0., 1., 1., 0., 1., 0., 1., 1.]])

In [8]:
# 수요 포인트 i의 야구장 수요 A

A = [i[1] for i in [*I.values()]]
A

[10, 40, 20, 20, 30, 10, 30, 50, 30, 10]

In [9]:
# 입지 후보지 j에서 기존 야구장 r까지의 거리 T

t_jr = np.array([])
for j_key, j_value in J.items():
    for r_key, r_value in R.items():
        # d = distance.euclidean(j_value[0], r_value[0])
        d = haversine_distances([[radians(_) for _ in j_value[0]], [
                                radians(_) for _ in r_value[0]]]) * 6371000/1000
        d = d[0][1]
        t_jr = np.append(t_jr, d)
t_jr = t_jr.reshape(len(J), len(R))

T = t_jr
pd.DataFrame(T)

,0,1,2,3,4
0,339.173377,647.917062,4670.925561,6166.253841,2474.882968
1,2669.722347,2263.562624,7006.079281,8469.815053,266.743701
2,2780.101841,2354.590896,7116.655512,8570.459659,173.605934
3,881.484477,1250.864311,3600.377354,5007.438215,3561.639085


In [41]:
# 입지 후보지 j에 야구장 설립 시 예상되는 수요

values = [ E[e].dot(A) for e in range(len(E)) ]
names = J.keys()
N = dict(zip(names, values))
N

{'강릉': 200.0, '춘천': 160.0, '진주': 160.0, '구미': 170.0}

# 2. Formulation

In [45]:
key_J, loc_J, demand_J = gp.multidict(J) # 각각 list, dict, dict

for key, value in demand_J.items():
    # do something with value
    demand_J[key] = N[key]
demand_J


{'강릉': 200.0, '춘천': 160.0, '진주': 160.0, '구미': 170.0}

In [46]:
# Declare and initialize model
m = gp.Model('Optimal Location')
m

<gurobi.Model Continuous instance Optimal Location: 0 constrs, 0 vars, No parameter changes>

In [58]:
loc_J.keys()

['강릉', '춘천', '진주', '구미']

In [47]:
# Create decision variables 
x = m.addVars(key_J, name="assign")
x

{'강릉': <gurobi.Var *Awaiting Model Update*>,
 '춘천': <gurobi.Var *Awaiting Model Update*>,
 '진주': <gurobi.Var *Awaiting Model Update*>,
 '구미': <gurobi.Var *Awaiting Model Update*>}

In [48]:
x.prod(demand_J)

<gurobi.LinExpr: 200.0 <gurobi.Var *Awaiting Model Update*> + 160.0 <gurobi.Var *Awaiting Model Update*> + 160.0 <gurobi.Var *Awaiting Model Update*> + 170.0 <gurobi.Var *Awaiting Model Update*>>

In [49]:
# Objective: maximize total matching score of all assignments
m.setObjective(x.prod(demand_J), GRB.MAXIMIZE)


In [52]:
# Save model for inspection
m.write('Optimal_Location.lp')
# m.display()


In [53]:
# Run optimization engine
m.optimize()


Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 0 rows, 4 columns and 0 nonzeros
Model fingerprint: 0xe3643a83
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [2e+02, 2e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.00s

Solved in 0 iterations and 0.00 seconds (0.00 work units)
Unbounded model


In [ ]:
# Display optimal values of decision variables
for v in m.getVars():
    if v.x > 1e-6:
        print(v.varName, v.x)

# Display optimal total matching score
print('Total matching score: ', m.objVal)
